In [3]:
import torchvision
import torch
from jetbot import Robot
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np
from IPython.display import display
import ipywidgets
import traitlets
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg
import math
import zmq
import time

ModuleNotFoundError: No module named 'jetbot'

In [2]:
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)
model.load_state_dict(torch.load('new3_best_steering.pth'))
device1 = torch.device('cuda')

model = model.to(device1)
model = model.eval().half()

In [3]:
model_collision = torchvision.models.alexnet(pretrained=False)
model_collision.classifier[6] = torch.nn.Linear(model_collision.classifier[6].in_features, 4)

model_collision.load_state_dict(torch.load('new0810_best_model.pth'))

device2 = torch.device('cuda')
model_collision = model_collision.to(device2)
model_collision = model_collision.eval().half()

In [4]:
mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device1).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [5]:
camera = Camera.instance(width=224, height=224, fps = 5)

In [6]:
robot = Robot(driver_board="dfrobot")

board begin success


In [4]:
speed_control_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01,value = 0.30, description='speed control')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.10, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.01, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')
display(speed_control_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider)


blocked_threshold= ipywidgets.FloatSlider(min=0, max=1.0, step=0.01, value=0.7, description='blocked threshold')
T_slider_threshold= ipywidgets.FloatSlider(min=0, max=1.0, step=0.01, value=0.8, description='T_slider threshold')

display(ipywidgets.HBox([blocked_threshold,T_slider_threshold]))


blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')
ITC_slider = widgets.FloatSlider(description='Intersection', min=0.0, max=1.0, orientation='vertical')
free_slider = widgets.FloatSlider(description='free', min=0.0, max=1.0, orientation='vertical')
T_slider = widgets.FloatSlider(description='T', min=0.0, max=1.0, orientation='vertical')

display(widgets.HBox([blocked_slider,free_slider,T_slider,ITC_slider]))

FloatSlider(value=0.3, description='speed control', max=1.0, step=0.01)

FloatSlider(value=0.1, description='steering gain', max=1.0, step=0.01)

FloatSlider(value=0.01, description='steering kd', max=0.5, step=0.001)

FloatSlider(value=0.0, description='steering bias', max=0.3, min=-0.3, step=0.01)

In [8]:
steer_val = 0
angle = 0.0
angle_last = 0.0
count_stops = 0
angle_subtrace = 0
go_on = 1
x = 0.0
y = 0.0
speed_value = speed_control_slider.value

def judgeCondition(change):#判斷前方狀況
    global prob_T, prob_blocked, prob_free, blocked_threshold, prob_ITC
    
    image_preproc2 = preprocess(change['new']).to(device2)
    prob_ITC  = float(F.softmax(model_collision(image_preproc2), dim=1).flatten()[0])
    prob_T = float(F.softmax(model_collision(image_preproc2), dim=1).flatten()[1])
    prob_blocked = float(F.softmax(model_collision(image_preproc2), dim=1).flatten()[2])
    prob_free = float(F.softmax(model_collision(image_preproc2), dim=1).flatten()[3])
    ITC_slider.value = prob_ITC
    T_slider.value = prob_T
    blocked_slider.value = prob_blocked
    free_slider.value = prob_free    
    steer_gain = steering_gain_slider.value
    steer_dgain = steering_dgain_slider.value
    steer_bias = steering_bias_slider.value
    
def execute(change):#road_Following改良演算法
    global angle, angle_last, blocked_slider, robot, x, y, steer_val
    global speed_value, steer_gain, steer_dgain, steer_bias, condition
    steer_gain = steering_gain_slider.value
    steer_dgain = steering_dgain_slider.value
    steer_bias = steering_bias_slider.value
    image_preproc1 = preprocess(change['new']).to(device1)
    angle = math.atan2(x, y)        
    if angle > -1.2 and angle < 1.2 :
        angle_subtrace = 0 
    else:
        angle_subtrace = angle
    xy = model(image_preproc1).detach().float().cpu().numpy().flatten()
    x = xy[0]            
    y = (0.5 - xy[1]) / 2.0
    speed_value = speed_control_slider.value
    pid = 2*(angle * steer_gain)/3 + (angle_subtrace) * steer_dgain
    steer_val = pid + steer_bias 
    angle_last = angle 

In [9]:
context = zmq.Context()
socket = context.socket(zmq.PUB)
socket.bind("tcp://192.168.100.108:5555")
socket.send(b"connecting")

In [10]:
def t_right(b):
    import time
    global steer_val,speed_value, socket, camera, prob_blocked
    if b['new']:#當按鈕按下時
        socket.send(b"Right")#傳送Right出去
        socket.send(b"On")#傳送On出去
        robot.left_motor.value = 0.4
        robot.right_motor.value = 0.4
        time.sleep(2.8)
        robot.left_motor.value = 0.5
        robot.right_motor.value = 0.0
        time.sleep(1.2)
        robot.left_motor.value = 0
        robot.right_motor.value = 0
        time.sleep(0.5)
        camera.observe(RC, names='value')#開啟相機更新自動執行
        
def t_left(b):
    import time
    global steer_val,speed_value, socket, camera, prob_blocked
    if b['new']:
        socket.send(b"Left")
        socket.send(b"On")
        robot.left_motor.value = 0.4
        robot.right_motor.value = 0.4
        time.sleep(2.8)
        robot.left_motor.value = 0.0
        robot.right_motor.value = 0.5
        time.sleep(1.2)
        robot.left_motor.value = 0
        robot.right_motor.value = 0
        time.sleep(0.5)
        
def start(b):
    global socket, camera
    if b['new']:                                #當按鈕按下時
        socket.send(b"On")                        #傳送On出去
        camera.observe(RC, names='value')#開啟相機更新自動執行
         
def break_RC(b):
    global camera, robot, socket
    if b['new']:                                     #當按鈕按下時
        socket.send(b"Off")                          #傳送Off出去
        camera.unobserve(RC, names = 'value')#關閉相機更新自動執行
        robot.stop()
        robot.left_motor.value =  0.0
        robot.right_motor.value = 0.0
        robot = Robot(driver_board="dfrobot")     #讓馬達數值歸0

def forward(b):
    import time
    global steer_val,speed_value, socket, camera, prob_blocked
    if b['new']:
        socket.send(b"Forward")
        socket.send(b"On")
        robot.left_motor.value =  0.5
        robot.right_motor.value = 0.5
        time.sleep(3)
        robot.stop()
        camera.observe(RC, names='value')

In [5]:
controller = widgets.Controller(index=0)  # replace with index of your controller
display(controller)

Controller()

In [14]:
controller.buttons[1].observe(t_right, names = 'value')
controller.buttons[2].observe(t_left, names = 'value')
controller.buttons[9].observe(start, names = 'value')
controller.buttons[0].observe(break_RC, names = 'value')
controller.buttons[3].observe(forward, names = 'value')
#手動控制馬達
left_link3 = traitlets.dlink((controller.axes[3], 'value'), (robot.left_motor,'value'), transform=lambda x: -0.5*x)
right_link3 = traitlets.dlink((controller.axes[3], 'value'), (robot.right_motor, 'value'), transform=lambda x: -0.5*x)
left_link0 = traitlets.dlink((controller.axes[0], 'value'), (robot.left_motor,'value'), transform=lambda x: 0.5*x)
right_link0 = traitlets.dlink((controller.axes[0], 'value'), (robot.right_motor, 'value'), transform=lambda x: -0.5*x)

In [15]:
def RC(change):
    global steer_val, speed_value, socket, camera, x, y, count, angle
    global prob_T, prob_blocked, prob_free, blocked_threshold, prob_ITC
    judgeCondition({'new': camera.value})
    execute({'new': camera.value})
    if prob_blocked > blocked_threshold.value:#障礙物
        socket.send(b"Waiting")
        robot.left_motor.value = int(robot.left_motor.value / 2)
        robot.right_motor.value = int(robot.left_motor.value / 2)
    elif prob_T > T_slider_threshold.value:  #T字路口
        x = 0.0 #set x steering to zero
        y = 0.0 #set y steering to zero
        speed_value = 0 # set speed to zero (can set to turn as well)
        camera.unobserve(RC, names = 'value')
        robot.stop()
        socket.send(b"T road")
    elif prob_ITC > 0.9:                     #十字路口
        x = 0.0 #set x steering to zero
        y = 0.0 #set y steering to zero
        camera.unobserve(RC, names = 'value')
        robot.stop()
        socket.send(b"Intersection")
        speed_value = 0 # set speed to zero (can set to turn as well)
    else:
        robot.left_motor.value = max(min(speed_value + steer_val, 1.0), 0.0)
        robot.right_motor.value = max(min(speed_value - steer_val, 1.0), 0.0)
        if angle > 1.2:                         #右轉
            socket.send(b"R")
        if angle < -1.2:                        #左轉
            socket.send(b"L")

In [16]:
RC({'new':camera.value}) #執行一次測試

In [17]:
camera.observe(RC, names='value')#相機更新時執行RC函式

In [18]:
camera.unobserve(RC, names='value')
robot.stop()

In [22]:
robot.left_motor.value = 0
robot.right_motor.value = 0
time.sleep(4)